In [1]:
import pandas as pd

import folium
import requests

from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

In [2]:
CLIENT_ID = 'HW2BMKCIJMFDHVHTRUGVFPB4XJL1U5EYVKOYW0BZ0FRRAX3V' # your Foursquare ID
CLIENT_SECRET = 'J2OZP1INRNJFY5TJDQDUNZDGIHCOLNUJAHUEV0ZPAXZKTUMR' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 100

In [3]:
df = pd.read_html('https://purlp.com/taguig/')[0]
Taguig_zip = df.iloc[1:]

Taguig_zip = Taguig_zip.rename(columns = Taguig_zip.iloc[0]).drop(Taguig_zip.index[0])
Taguig_zip = Taguig_zip[['Postal Codes', 'Locations']]


Taguig_zip.head()

,Postal Codes,Locations
2,1630,Bagumbayan; Pinagsama (including EP Village)
3,1636,Bay Breeze Executive Village
4,1636,"Maharlika Village, Taguig"
5,1636,"Tanyag, Taguig"
6,1631,Bicutan


In [4]:
ph_zip = pd.read_csv('C:\\Users\\alfre\\Documents\\env\\IBM_Capstone\\Coursera_Capstone\\PH.csv', sep=',', encoding='latin-1', header=0)
ph_zip.head()

,Zip Code,Neighborhood,Latitude,Longitude
0,1630,Bagumbayan; Pinagsama (including EP Village),14.517618,121.048676
1,1636,Bay Breeze Executive Village,14.512020,121.078615
2,1636,"Maharlika Village, Taguig",14.498195,121.051167
3,1636,"Tanyag, Taguig",14.476468,121.044121
4,1631,Bicutan,14.497467,121.045143


In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
# Select first building (which in our case is BedokVille):
neighborhood_name = ph_zip.loc[0, 'Neighborhood']
neighborhood_latitude = ph_zip.loc[0, 'Latitude'] 
neighborhood_longitude = ph_zip.loc[0, 'Longitude'] 
# limit of number of venues returned by Foursquare API
LIMIT = 100 
radius = 500

In [7]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [8]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)

In [9]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,McDonald's,Fast Food Restaurant,14.516306,121.051504
1,C5 Waterfun,Sculpture Garden,14.516760,121.051396
2,Heritage Park Jogging Oval,Park,14.519163,121.049195
3,Kuyang Food Trip ng Pinoy Head Office,Food Truck,14.514962,121.048027
4,Housing Wet Market,Market,14.518201,121.052000


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
venues_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)
# add a red circle marker to represent the BEDOKVILLE
folium.CircleMarker(
    [neighborhood_latitude, neighborhood_longitude],
    radius=10,
    color='red',
    popup='BEDOKVILLE',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
# add all venues as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

In [12]:
venues_map

In [13]:
taguig_venues = getNearbyVenues(names=ph_zip['Neighborhood'],
                                   latitudes=ph_zip['Latitude'],
                                   longitudes=ph_zip['Longitude']
                                  )

Bagumbayan; Pinagsama (including EP Village)
Bay Breeze Executive Village
Maharlika Village, Taguig
Tanyag, Taguig
Bicutan
Hagonoy
Lower Bicutan  Ususan
New Lower Bicutan, Taguig
North Daang Hari, Taguig
San Miguel, Taguig
Ususan, Taguig
South Daang Hari, Taguig
Wawa, Taguig
Central Bicutan, Taguig
Upper Bicutan
Central Signal Village
Western Bicutan (including Food Terminal, Inc....
Ligid
Nichols  McKinley Hill (Fort Bonifacio, Globa...
Tuktukan
Bambang
Calzada
Ibayo-Tipas
Ligid-Tipas
Napindan, Taguig
Palingon, Taguig
Santa Ana, Taguig
Tuktukan, Taguig
Tilapya
Pinagsama, Taguig
Ususan and Palar Village


In [14]:
taguig_venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bagumbayan; Pinagsama (including EP Village),14.517618,121.048676,McDonald's,14.516306,121.051504,Fast Food Restaurant
1,Bagumbayan; Pinagsama (including EP Village),14.517618,121.048676,C5 Waterfun,14.516760,121.051396,Sculpture Garden
2,Bagumbayan; Pinagsama (including EP Village),14.517618,121.048676,Heritage Park Jogging Oval,14.519163,121.049195,Park
3,Bagumbayan; Pinagsama (including EP Village),14.517618,121.048676,Kuyang Food Trip ng Pinoy Head Office,14.514962,121.048027,Food Truck
4,Bagumbayan; Pinagsama (including EP Village),14.517618,121.048676,Housing Wet Market,14.518201,121.052000,Market
5,Bay Breeze Executive Village,14.512020,121.078615,NAIA Canteen,14.509831,121.080218,Cafeteria
6,Bay Breeze Executive Village,14.512020,121.078615,Siomai House,14.514723,121.081629,Food Truck
7,Bay Breeze Executive Village,14.512020,121.078615,Phines Bakery,14.516211,121.077801,Bakery
8,Bay Breeze Executive Village,14.512020,121.078615,All Day Supermarket,14.512756,121.074039,Market
9,"Maharlika Village, Taguig",14.498195,121.051167,Ayala Malls Arca South,14.496085,121.047928,Shopping Mall
